In [1]:
import os
import numpy as np
import torch as tr

from CSWSEM import *
from matplotlib import pyplot as plt

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
task = CSWTask()

exp_kwargs={
    'condition':'blocked',
    'n_train':8,
    'n_test':2
}
exp = task.generate_experiment(**exp_kwargs)

stsize = 25
learn_rate = 0.05
seed = 97
alfa = 10
lmda = 1
nosplit = False

sem_kwargs={
    'nosplit':nosplit,
    'stsize':stsize,
    'lmda':lmda,
    'alfa':alfa,
    'learn_rate':learn_rate,
    'seed':seed,
}

sem = SEM(**sem_kwargs)
# sem.forward_trial(event)
sem_data = sem.forward_exp(exp)
sem_data



[{'trial': 0,
  'prior': array([2.30258509, 2.30258509]),
  'like': array([-44.18758382, -44.18758382]),
  'active_schema': 0,
  'loss': array(0.60795504, dtype=float32),
  'nosplit': False,
  'stsize': 25,
  'learn_rate': 0.05,
  'lmda': 1,
  'alfa': 10,
  'seed': 97},
 {'trial': 1,
  'prior': array([1.94591015, 2.30258509]),
  'like': array([-132.42861093,  -44.18758382]),
  'active_schema': 1,
  'loss': array(0.71737033, dtype=float32),
  'nosplit': False,
  'stsize': 25,
  'learn_rate': 0.05,
  'lmda': 1,
  'alfa': 10,
  'seed': 97},
 {'trial': 2,
  'prior': array([1.79175947, 1.79175947, 2.30258509]),
  'like': array([-238.65232151, -169.93174568,  -34.78639788]),
  'active_schema': 2,
  'loss': array(0.5972318, dtype=float32),
  'nosplit': False,
  'stsize': 25,
  'learn_rate': 0.05,
  'lmda': 1,
  'alfa': 10,
  'seed': 97},
 {'trial': 3,
  'prior': array([1.79175947, 1.79175947, 1.79175947, 2.30258509]),
  'like': array([-229.44740871, -124.92222868, -254.76962545,  -39.35201779

In [4]:
import pandas as pd
pd.DataFrame(sem_data)

,trial,prior,like,active_schema,loss,nosplit,stsize,learn_rate,lmda,alfa,seed
0,0,"[2.302585092994046, 2.302585092994046]","[-44.1875838216743, -44.1875838216743]",0,0.60795504,False,25,0.05,1,10,97
1,1,"[1.9459101490553132, 2.302585092994046]","[-132.42861092846158, -44.1875838216743]",1,0.71737033,False,25,0.05,1,10,97
2,2,"[1.791759469228055, 1.791759469228055, 2.30258...","[-238.65232151257493, -169.93174568344037, -34...",2,0.5972318,False,25,0.05,1,10,97
3,3,"[1.791759469228055, 1.791759469228055, 1.79175...","[-229.44740871232287, -124.92222868375873, -25...",3,0.57549536,False,25,0.05,1,10,97
4,4,"[1.791759469228055, 1.791759469228055, 1.79175...","[-255.28254440637403, -199.14668851327423, -31...",4,0.696276,False,25,0.05,1,10,97
5,5,"[1.791759469228055, 1.791759469228055, 1.79175...","[-255.28254440637403, -199.14668851327423, -31...",5,0.696276,False,25,0.05,1,10,97
6,6,"[1.791759469228055, 1.791759469228055, 1.79175...","[-229.44740871232287, -124.92222868375873, -25...",6,0.57549536,False,25,0.05,1,10,97
7,7,"[1.791759469228055, 1.791759469228055, 1.79175...","[-238.65232151257493, -169.93174568344037, -19...",7,0.5972318,False,25,0.05,1,10,97
8,8,"[1.791759469228055, 1.791759469228055, 1.79175...","[-238.65232151257493, -169.93174568344037, -19...",8,0.5972318,False,25,0.05,1,10,97
9,9,"[1.791759469228055, 1.791759469228055, 1.79175...","[-255.28254440637403, -199.14668851327423, -31...",9,0.696276,False,25,0.05,1,10,97


In [ ]:

plt.plot(1-pd.DataFrame(sem_data).loss)